<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.2
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerous than some, and does not have waiting period for approval of new developers.

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access
- fill in the form fields at the bottom
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"

- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 8.1 MB/s eta 0:00:00


In [14]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time
import warnings
warnings.filterwarnings("ignore")

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [ ]:
my_user_agent = ''   # your user Agent string goes in here
my_client_id = ''   # your Client ID string goes in here
my_client_secret = ''   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [2]:
pwd()  # make sure your working directory is where the file is

'/Users/veralyu/Desktop/iod/Module 3/Labs 3'

In [5]:
data = {
    "my_client_id": 'k_hjP1d-mZayccejDIYwNQ',
    "my_client_secret": 'c7pHgPBAiP4RWRvE4lM8Br_UF5R0nw',
    "my_user_agent": 'Jupiter_Notebooks-python:Walk_and_Talk:v1.0 (by /u/Walk_and_Talk)'
}

# Specify the file path where you want to save the JSON file
file_path = "auth_reddit.json"

# Write the data to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"JSON data has been written to {file_path}")

JSON data has been written to auth_reddit.json


In [6]:
path_auth = 'auth_reddit.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:

pp.pprint(auth)




{   'my_client_id': 'k_hjP1d-mZayccejDIYwNQ',
    'my_client_secret': 'c7pHgPBAiP4RWRvE4lM8Br_UF5R0nw',
    'my_user_agent': 'Jupiter_Notebooks-python:Walk_and_Talk:v1.0 (by '
                     '/u/Walk_and_Talk)'}


In [7]:
my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']

Security considerations:
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [8]:
reddit = praw.Reddit(client_id = my_client_id,
                     client_secret = my_client_secret,
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [9]:
subreddit_name = 'malaysia'
subreddit = reddit.subreddit(subreddit_name)

In [15]:
comments = []
for comment in subreddit.comments(limit=1000):
    comments.append(comment)

In [16]:
reddit.subreddit(subreddit_name)

Subreddit(display_name='malaysia')

In [17]:
reddit.comment

<bound method Reddit.comment of <praw.reddit.Reddit object at 0x111c23350>>

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic.

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [21]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
Completed my first real Python project!!
Why python chose def instead of fun or func?
Why is my function not returning anything when I run it?
While loop sending prompt one more time than I think it should
Having a hard time with sets
Best way to backup projects
How do you run programs when away from your computer?
Check for same number of (unquoted) delimiters in each row of a csv with Pandas?
Print dataclasses with indendation levels?


Now retrieve 10 authors:

In [22]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
lelel04
conceptcreatormiui
Cartoony_Sam
WarsWorth
SevenHundredAnd77
ironman730
PremeJigg
PangeanPrawn
51dux


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [23]:
submissions=reddit.subreddit('learnpython')

Why doesn't the next cell produce output?

In [24]:
for submission in submissions:
    print(submission.comments)

TypeError: 'Subreddit' object is not iterable

Print two comments associated with each of these submissions:

In [25]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

Why is re.sub() not substituting when I have long list of IP pairs which I need to change?

I have a list which has many rows of IP pairs. Single row is IP-address <tab> IP-address. 

Ie:

10.32.2.8	10.99.99.228  
10.32.2.9	10.99.99.229  
10.32.2.10	10.99.99.230  
10.32.2.11	10.99.99.231

This file is used as the file where the IP's needs to be change and also as the file where I get the new IP which is replacing the old. So Im searching the first IP and replacing it with the second. Making this a bit simplier than what it is where I need to use it.

I'm trying to replace the first IP with the second one, but re.sub() seems to fails at some point depending on how long the list is.

My code is this (newIPs and data is read from same file and example of the file is above):

`for ip in enumerate(newIPs):`  
`helpVar = ip[1].split("\t")`  
`search_result = re.search(helpVar[0],data)`  
`print("Looking for: ",helpVar[0],", search_result:",search_result, "Replacing with:", helpVar[1])`  
`da

Referring to the API documentation, explore the submissions object and print some interesting data:

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request:

In [26]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

False


You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

>
>


>
>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



